# Ch11: Dictionaries

### A Dictionary is Mapping

A dictionary is like an array, but more general. In an array, the indices have to be integers; in a dictionary they can be (almost) any type. 

A dictionary contains a collection of indices, which are called keys, and a collection of values. Each key is associated with a single value. The association of a key and a value is called a key-value pair or sometimes an item. In mathematical language, a dictionary represents a mapping from keys to values, so you can also say that each key “maps to” a value. As an example, we’ll build a dictionary that maps from English to Spanish words, so the keys and the values are all strings.


In [1]:
eng2sp = Dict()

Dict{Any,Any} with 0 entries

To add items to the dictionary, you can use square brackets

In [3]:
eng2sp["one"] = "uno"

"uno"

In [4]:
eng2sp

Dict{Any,Any} with 1 entry:
  "one" => "uno"

To create a dictionary with multiple entries, we use arrow notations when we call `Dict`

In [6]:
eng2sp = Dict("one" => "uno", "two" => "dos", "three" => "tres")

Dict{String,String} with 3 entries:
  "two"   => "dos"
  "one"   => "uno"
  "three" => "tres"

The order of the key-value pairs might not be the same. If you type the same example on your computer, you might get a different result. In general, the order of items in a dictionary is unpredictable.

The length function works on dictionaries; it returns the number of key-value pairs.

In [7]:
length(eng2sp)

3

The function `keys` returns a collection with the keys of the dictionary:

In [8]:
ks = keys(eng2sp)
print(ks)

["two", "one", "three"]

Now you can use the ∈ operator to see whether something appears as a key in the dictionary

In [9]:
"one" ∈ ks, "uno" ∈ ks

(true, false)

To see whether something appears as a value in a dictionary, you can use the function `values`, which returns a collection
of values, and then use the ∈ operator:

In [10]:
vs = values(eng2sp);
"uno" ∈ vs

true

The ∈ operator uses different algorithms for arrays and dictionaries. For arrays, it searches the elements of the array in order. As the array gets longer, the search time gets longer in direct proportion.

For dictionaries, Julia uses an algorithm called a hash table that has a remarkable property: the ∈ operator takes about the same amount of time no matter how many items are in the dictionary.

A `hash` is a function that takes a value (of any kind) and returns an integer. Dictionaries use these integers, called hash values, to store and look up key-value pairs.


### Dictionary as a Collection of Counters

Suppose you are given a string and you want to count how many times each letter appears. There are several ways you
could do it:

- You could create 26 variables, one for each letter of the alphabet. Then you could traverse the string and, for each character, increment the corresponding counter, probably using a chained conditional.
- You could create an array with 26 elements. Then you could convert each character to a number (using the built-in function Int), use the number as an index into the array, and increment the appropriate counter.
- You could create a dictionary with characters as keys and counters as the corresponding values. The first time you see a character, you would add an item to the dictionary. After that you would increment the value of an existing item. 

Each of these options performs the same computation, but each of them implements that computation in a different way. An implementation is a way of performing a computation; some implementations are better than others. For example, an advantage of the dictionary implementation is that we don’t have to know ahead of time which letters appear in the string and we only have to make room for the letters that do appear.

In [11]:
function histogram(s) 
    d = Dict()
    for c in s
        if c ∉ keys(d)
            d[c] = 1
        else
            d[c] += 1
        end
    end
    d
end

histogram (generic function with 1 method)

In [12]:
h = histogram("brontosaurus")

Dict{Any,Any} with 8 entries:
  'n' => 1
  's' => 2
  'a' => 1
  'r' => 2
  't' => 1
  'o' => 2
  'u' => 2
  'b' => 1

Dictionaries have a function called `get` that takes a key and a default value. If the key appears in the dictionary, `get`
returns the corresponding value; otherwise it returns the default value. For example:

In [13]:
h = histogram("a")

get(h, 'a', 0), get(h, 'b', 0)

(1, 0)

### Exercise 11-1

Use `get` to write `histogram` more concisely. You should be able to eliminate the `if` statement.

In [14]:
function histogram(s) 
    d = Dict()
    for c in s
        d[c] = get(d,c,0) + 1
    end
    d
end

histogram (generic function with 1 method)

In [15]:
h = histogram("brontosaurus")

Dict{Any,Any} with 8 entries:
  'n' => 1
  's' => 2
  'a' => 1
  'r' => 2
  't' => 1
  'o' => 2
  'u' => 2
  'b' => 1

### Looping and Dictionaries

You can traverse the keys of the dictionary in a `for` statement. For example, `printhist` prints each key and the corresponding value:

In [16]:
function printhist(h)
    for c in keys(h)
        println(c, " ", h[c])
    end
end

printhist (generic function with 1 method)

In [18]:
h = histogram("parrot")

Dict{Any,Any} with 5 entries:
  'a' => 1
  'r' => 2
  'p' => 1
  'o' => 1
  't' => 1

In [19]:
printhist(h)

a 1
r 2
p 1
o 1
t 1


Once again, the keys are in no particular order. To traverse the keys in sorted order, you can combine `sort` and `collect`:

In [20]:
for c in sort(collect(keys(h)))
    println(c, " ", h[c])
end

a 1
o 1
p 1
r 2
t 1


### Reverse Lookup

Given a dictionary `d` and a key `k` , it is easy to find the corresponding value `v = d[k]`. This operation is called a lookup.  But what if you have v and you want to find k ? You have two problems: first, there might be more than one key that maps to the value v . Depending on the application, you might be able to pick one, or you might have to make an array that contains all of them. Second, there is no simple syntax to do a reverse lookup; you have to search.

In [21]:
function reverselookup(d, v)
    for k in keys(d)
        if d[k] == v
            return k
        end
    end
    error("LookupError")  # throws an exception if not found
end

reverselookup (generic function with 1 method)

In [22]:
h = histogram("parrot");
key = reverselookup(h, 2)

'r': ASCII/Unicode U+0072 (category Ll: Letter, lowercase)

In [23]:
key = reverselookup(h, 3)

ErrorException: LookupError

**Julia provides an optimized way to do a reverse lookup: `findall(isequal(3), h)`.**

A reverse lookup is much slower than a forward lookup; if you have to do it often, or if the dictionary gets big, the performance of your program will suffer.

### Dictionaries and Arrays

Arrays can appear as values in a dictionary. For example, if you are given a dictionary that maps from letters to frequencies, you might want to invert it; that is, create a dictionary that maps from frequencies to letters. Since there might be several letters with the same frequency, each value in the inverted dictionary should be an array of letters. Here is a function that inverts a dictionary:

In [25]:
function invertdict(d)
    inverse = Dict()
    for key in keys(d)
        val = d[key]
        if val ∉ keys(inverse)
            inverse[val] = [key]   # an array with one element is called singleton
        else
            push!(inverse[val], key)  # each item in the dictionary is an array
        end
    end
    inverse
end

invertdict (generic function with 1 method)

In [26]:
hist = histogram("parrot");
inverse = invertdict(hist)

Dict{Any,Any} with 2 entries:
  2 => ['r']
  1 => ['a', 'p', 'o', 't']

### Memos

If you played with the fibonacci function from One More Example, you might have noticed that the bigger the argument you provide, the longer the function takes to run. Furthermore, the run time increases quickly.

A call graph for `fibonacci` with a value like `n` = 4 shows that `fibonacci(0)` and `fibonacci(1)` would be called many times when `n` is large.

One solution is to keep track of values that have already been computed by storing them in a dictionary. A previously computed value that is stored for later use is called a memo. Here is a “memoized” version of fibonacci:



In [27]:
known = Dict(0=>0, 1=>1)

function fibonacci(n)
    if n ∈ keys(known)
        return known[n]
    end
    res = fibonacci(n-1) + fibonacci(n-2)
    known[n] = res
    res
end

fibonacci (generic function with 1 method)

In [29]:
fibonacci(4)

3

In [30]:
fibonacci(10)

55

### Global Variables

In the previous example, known is created outside the function, so it belongs to the special frame called Main . Variables in Main are sometimes called global because they can be accessed from any function. Unlike local variables, which disappear when their function ends, global variables persist from one function call to the next. It is common to use global variables for flags; that is, boolean variables that indicate (“flag”) whether a condition is true. 

For example, some programs use a flag named verbose to control the level of detail in the output:

In [31]:
verbose = true

function example1() 
    if verbose
        println("Running example1")
    end
end

example1 (generic function with 1 method)

In [32]:
example1()

Running example1


To reassign a global variable inside a function you have to declare the variable global before you use it.  The global statement tells the interpreter something like, “In this function, when I say been_called , I mean the global variable; don’t create a local one.” Here’s an example that tries to update a global variable:

In [33]:
count = 0
function example3()
    count = count + 1 # WRONG
end

example3 (generic function with 1 method)

In [34]:
example3()

UndefVarError: UndefVarError: count not defined

Inside the function, Julia creates a new local variable `count`.  Julia therefore thinks that you are reading it before writing it. The solution is to declare `count` global.

In [35]:
count = 0
function example3()
    global count
    count += 1
end

example3 (generic function with 1 method)

In [36]:
example3()

1

There is one exception.  If a global variable refers to a mutable value, you can modify the value without declaring the variable global:

In [37]:
known = Dict(0=>0, 1=>1)
function example4()
    known[2] = 1
end

example4 (generic function with 1 method)

In [38]:
example4();
known

Dict{Int64,Int64} with 3 entries:
  0 => 0
  2 => 1
  1 => 1

So you can add, remove and replace elements of a global array or dictionary.  **But if you want to reassign the array or dictionary variable, you still have to declare it global:**

In [42]:
known = Dict(0=>0, 1=>1)

function example5()
    global known
    known = Dict()
end

example5 (generic function with 1 method)

For performance reasons, you should declare a global variable constant. You can no longer reassign the variable but if it
refers to a mutable value, you can modify the value.

In [41]:
const known2 = Dict(0=>0, 1=>1)

function example4()
    known2[2] = 1
end

example4 (generic function with 1 method)

Global variables can be useful, but if you have a lot of them, and you modify them frequently, they can make programs hard to debug and perform badly.

### Debugging Tips for Big Datasets

As you work with bigger datasets it can become unwieldy to debug by printing and checking the output by hand. Here are some suggestions for debugging large datasets:

* Scale down the input: If possible, reduce the size of the dataset. For example if the program reads a text file, start with just the first 10 lines, or with the smallest example you can find which errors. You sould not edit the files themselves, but rather modify the program so it reads only the first lines. If there is an error, you can reduce to the smallest value that manifests the error, and then increase it gradually as you find and correct errors.

* Check summaries and types: Instead of printing and checking the entire dataset, consider printing summaries of the data: for example, the number of items in a dictionary or the total of an array of numbers. A common cause of runtime errors is a value that is not the right type. For debugging this kind of error, it is often enough to print the type of a value.

* Write self-checks: Sometimes you can write code to check for errors automatically. For example, if you are computing the average of an array of numbers, you could check that the result is not greater than the largest element in the array or less than the smallest. This is called a “sanity check”. Another kind of check compares the results of two different computations to see if they are consistent. This is called a “consistency check”.

* Format the output: Formatting debugging output can make it easier to spot an error. We saw an example in Debugging. Again, time you spend building scaffolding can reduce the time you spend debugging.

### Exercise 11-2

Write a function that reads the words in words.txt and stores them as keys in a dictionary. It doesn’t matter what the values are. Then you can use the ∈ operator as a fast way to check whether a string is in the dictionary. If you did Exercise 10-10, you can compare the speed of this implementation with the array ∈ operator and the bisection search.


In [45]:
function build_words_dict()
    
    dict = Dict();
    
    for word in eachline("words.txt")
        dict[word] = ""
    end

    return dict
    
end

words_dict = build_words_dict();

In [47]:
"zebra" ∈ keys(words_dict)

true

In [48]:
"bookkeeper" ∈ keys(words_dict)

true

### Exercise 11-3

Read the documentation of the dictionary function `get!` and use it to write a more concise version of `invertdict`.

In [49]:
?get!

search: get! get getpid getkey getfield getindex getproperty gethostname



```
get!(collection, key, default)
```

Return the value stored for the given key, or if no mapping for the key is present, store `key => default`, and return `default`.

# Examples

```jldoctest
julia> d = Dict("a"=>1, "b"=>2, "c"=>3);

julia> get!(d, "a", 5)
1

julia> get!(d, "d", 4)
4

julia> d
Dict{String,Int64} with 4 entries:
  "c" => 3
  "b" => 2
  "a" => 1
  "d" => 4
```

---

```
get!(f::Function, collection, key)
```

Return the value stored for the given key, or if no mapping for the key is present, store `key => f()`, and return `f()`.

This is intended to be called using `do` block syntax:

```julia
get!(dict, key) do
    # default value calculated here
    time()
end
```


In [50]:
# function invertdict(d)
#     inverse = Dict()
#     for key in keys(d)
#         val = d[key]
#         if val ∉ keys(inverse)
#             inverse[val] = [key]   # an array with one element is called singleton
#         else
#             push!(inverse[val], key)  # each item in the dictionary is an array
#         end
#     end
#     inverse
# end


function invertdictV2(d)
    inverse = Dict()
    
    for key in keys(d)
              
        val = d[key]
        push!( get!(inverse,val,[]), key )
        
    end
    
    inverse
    
end

invertdictV2 (generic function with 1 method)

In [51]:
hist = histogram("parrot");
inverse = invertdictV2(hist)

Dict{Any,Any} with 2 entries:
  2 => Any['r']
  1 => Any['a', 'p', 'o', 't']

### Exercise 11-4

Memoize the `Ackermann` function from Exercise 6-5 and see if memoization makes it possible to evaluate the function with
bigger arguments.

In [52]:
# function ack(m,n)
    
#     if m == 0
#       #  @show m,n
#         return n+1
#     elseif m > 0 && n == 0
#       #  @show m,n
#         return ack(m-1,1)
#     elseif m > 0 && n > 0
#         return ack(m-1,ack(m,n-1))
#     end
    
# end


# known = Dict(0=>0, 1=>1)

# function fibonacci(n)
#     if n ∈ keys(known)
#         return known[n]
#     end
#     res = fibonacci(n-1) + fibonacci(n-2)
#     known[n] = res
#     res
# end

known = Dict()

function ack(m,n)
    
    if (m,n) ∈ keys(known)        # tuples can be keys in a dictionary
        return known[(m,n)]
    end
    
    if m == 0
        res = n+1
    elseif m > 0 && n == 0
        res = ack(m-1,1)
    elseif m > 0 && n > 0
        res = ack(m-1,ack(m,n-1))
    end
    
    known[(m,n)] = res
    return res
    
end

ack (generic function with 1 method)

In [53]:
ack(3,4)

125

In [57]:
ack(4,1), ack(5,0)

(65533, 65533)

In [58]:
ack(4,2)

StackOverflowError: StackOverflowError:

### Exercise 11-5

If you did Exercise 10-7, you already have a function named `hasduplicates` that takes an array as a parameter and returns true if there is any object that appears more than once in the array. Use a dictionary to write a faster, simpler version of `hasduplicates.

In [59]:
# function hasduplicates(arr)
    
#     for index in eachindex(arr[1:end-1])
        
#         if arr[index] ∈ arr[index+1:end]
#              return true
#         end
        
#     end
    
#     return false
    
# end

function hasduplicatesVDic(arr)
    
    d = Dict()
    for item in arr
        d[item] = get(d,item,0) + 1
        if d[item] >= 2
            return true
        end
    end

    false
    
end   


hasduplicatesVDic (generic function with 1 method)

In [61]:
hasduplicatesVDic([1,2,3,4,5,6,7,8,9,10]), hasduplicatesVDic([1,2,3,4,5,6,7,8,9,1])

(false, true)

### Exercise 11-6

Two words are “rotate pairs” if you can rotate one of them and get the other (see rotateword in Exercise 8-11). Write a program that reads a word array and finds all the rotate pairs.

In [74]:
function rotateword(input_string, shift_amount)
    
    rotated_string = ""
    
    for c in input_string
    
        if islowercase(c)
            offset = Int('a')
        else
            offset = Int('A')
        end
        
        new_char = Char( ( (Int(c)-offset+shift_amount)%26 + 26 )%26 + offset)
        
        rotated_string = rotated_string * new_char
        
    end
    
    return rotated_string
    
end


rotate_pairs=[]
for key in keys(words_dict)
    
     for i = -25:25
        if i != 0
            rotated = rotateword(key,i)
            if rotated ∈ keys(words_dict)
                push!(rotate_pairs,(key,rotated,i))
            end
        end
    end
    
end

length(rotate_pairs)

4360

In [73]:
rotate_pairs[500]

("kiva", "cans", 18)

### Exercise 11-7

Here’s another Puzzler from Car Talk (https://www.cartalk.com/puzzler/browse):

"This was sent in by a fellow named Dan O’Leary. He came upon a common one-syllable, five-letter word recently that has the following unique property. When you remove the first letter, the remaining letters form a homophone of the original word, that is a word that sounds exactly the same. Replace the first letter, that is, put it back and remove the second letter and the result is yet another homophone of the original word. And the question is, what’s the word?

Now I’m going to give you an example that doesn’t work. Let’s look at the five-letter word, ‘wrack.’ W-R-A-C-K, you know like to ‘wrack with pain.’ If I remove the first letter, I am left with a fourletter word, ’R-A-C-K.’ As in, ‘Holy cow, did you see the rack on that buck! It must have been a ninepointer!’ It’s a perfect homophone. If you put the ‘w’ back, and remove the ‘r,’ instead, you’re left with the word, ‘wack,’ which is a real word, it’s just not a homophone of the other two words. But there is, however, at least one word that Dan and we know of, which will yield two homophones if you remove either of the first two letters to make two, new four-letter words. The question is, what’s the word?"

You can use the dictionary from Exercise 11-2 to check whether a string is in the word array.

To check whether two words are homophones, you can use the CMU Pronouncing Dictionary. You can download it from http://www.speech.cs.cmu.edu/cgi-bin/cmudict.

Write a program that lists all the words that solve the Puzzler.

TIP: First build a dictionary with the words as the keys and their pronunciations as the values.  Note that the CMU Pronouncing Dictionary has all items capitalized.  Also, it appears that some words in this pronunciation dictionary do not appear in our main word dictionary.  We will go with the main dictionary.

In [108]:
function build_pronun_dict()
    
    dict = Dict();
    
    for string in eachline("cmudict-rem-punc.txt")
        aftersplit = split(string)
        word = popfirst!(aftersplit)
        pronunc = join(aftersplit,' ')
        dict[word] = pronunc
    end

    return dict
    
end


mypronundict = build_pronun_dict();


To be picky, it also lists multiple alternative pronunciations for some of the words ((1), (2), etc).  But, let's ignore that for this exercise.

In [124]:
mypronundict["PRAN"],mypronundict["PRAN(1)"]



("P R AA1 N", "P R AE1 N")

In [125]:
for key in keys(words_dict)
    
    if length(key) == 5
        
        word1 = key
        word2 = word1[2:end]
        word3 = word1[1] * word1[3:end]
                      
        if !(word1 ∈ keys(words_dict) && word2 ∈ keys(words_dict) && word3 ∈ keys(words_dict))
            continue
        end
            
        if ( get(mypronundict,uppercase(word1),"1") ==      # In case any of the words are not in mypronundict,
                get(mypronundict,uppercase(word2),"2") ==   # we set get to output random different values "1", "2", and "3"
                 get(mypronundict,uppercase(word3),"3") )   # We could have used another if statement.
            
                println(word1)
            
        end
            
        
    end
    
end


scent
llama
